## Sequential Chain

https://js.langchain.com/docs/modules/chains/

In [1]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [2]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(temperature=0.9)

In [3]:
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to enlish:"
    "\n\n {Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(
    llm = llm,
    prompt = first_prompt,
    output_key = 'English_Review'
    )

In [4]:
# prompt template 2: summarise the review english
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence"
    "\n\n{English_Review}"
)

# chain 2: input = English Review output = summary
chain_two = LLMChain(
    llm = llm,
    prompt = second_prompt,
    output_key = 'summary'
)

In [5]:
# prompt template 3: detect the language of review
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)

# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )

In [6]:
# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)

# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )


### Review Data Frame

In [7]:
import pandas as pd
df = pd.read_csv('Data.csv')

In [8]:
df

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...
5,L'Or Espresso Café \n,Je trouve le goût médiocre. La mousse ne tient...
6,Hervidor de Agua Eléctrico,"Está lu bonita calienta muy rápido, es muy fun..."


In [34]:
# overall_chain: input= Review 
# and output= English_Review,summary, language, followup_message
from langchain.chains import SequentialChain
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","language","followup_message"],
    verbose=True
)

In [35]:
review = df.Review[6]
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'Review': 'Está lu bonita calienta muy rápido, es muy funcional, solo falta ver cuánto dura, solo llevo 3 días en funcionamiento.',
 'English_Review': '"This beautiful stove heats up very quickly, it is very functional, we just need to see how long it lasts, I have only been using it for 3 days."',
 'summary': 'The stove is functional and heats up quickly, but the reviewer cannot comment on its longevity yet.',
 'language': 'Spanish',
 'followup_message': 'Resumen: La estufa funciona bien y se calienta rápidamente, pero el revisor aún no puede comentar sobre su longevidad.\n\nRespuesta: Es bueno saber que la estufa funciona correctamente y se calienta rápidamente. Sin embargo, es importante tener en cuenta la longevidad y espero que pueda proporcionar una actualización en el futuro. Asegúrese de seguir las instrucciones de cuidado y mantenimiento para ayudar a mantener la estufa en buen estado durante mucho tiempo.'}